# Bash Tools

> Tools for executing bash commands and searching files safely.

In [ ]:
# | default_exp bash_tools

In [ ]:
# | export
import os
import re
import shlex
import subprocess
from pathlib import Path
from typing import Dict, Any, List, Optional

## Safety Constants

These lists help ensure commands are executed safely.

In [ ]:
# | export
DEFAULT_DENY_LIST = [
    "rm -rf", "sudo", "chmod 777", ":(){ :|:& };:",
    "> /dev/sda", "dd if=/dev/zero", "mkfs", "> /dev/null",
    "mv /* ", "shred", "> /etc/passwd", "wget -O- | sh"
]

In [ ]:
# | export
SAFE_COMMANDS = [
    "ls", "cat", "echo", "pwd", "cd", "grep", "find",
    "head", "tail", "wc", "file", "mkdir", "touch",
    "diff", "cp", "mv", "du", "df", "ping", "ps"
]

## Command Execution

The `execute` function provides a safe way to run shell commands with explicit allow/deny controls.

In [ ]:
# | export
def execute(
    command: str,                                    # Shell command to run
    timeout: int = 30,                               # Maximum execution time in seconds
    allow_list: Optional[List[str]] = None,          # If provided, ONLY these commands will be allowed
    deny_list: Optional[List[str]] = DEFAULT_DENY_LIST  # Commands that will be explicitly blocked
) -> str:
    """Execute a shell command with safety controls via allow/deny lists.
    
    Args:
        command: Shell command to run
        timeout: Maximum execution time in seconds
        allow_list: If provided, ONLY these commands will be allowed
        deny_list: Commands that will be explicitly blocked
    
    Returns:
        Command output or error message
    """
    # Parse command to get the base command
    try:
        cmd_parts = shlex.split(command)
        if not cmd_parts: return "Error: Empty command"
        base_cmd = cmd_parts[0]
        
        # Check allow list if provided
        if allow_list is not None:
            if base_cmd not in allow_list:
                return f"Error: Command '{base_cmd}' not in allowed list. Allowed: {', '.join(allow_list)}"
        
        # Check deny list for exact matches and patterns
        if deny_list:
            for denied in deny_list:
                if command.startswith(denied) or f" {denied}" in command:
                    return f"Error: Command contains denied pattern '{denied}'"
        
        # Execute if safe
        result = subprocess.run(
            command,
            shell=True,  # We need shell=True to run full commands with pipes, etc.
            capture_output=True,
            text=True,
            timeout=timeout
        )
        
        output = result.stdout
        error = result.stderr
        
        if result.returncode != 0:
            return f"Command failed with exit code {result.returncode}.\nError:\n{error}\nOutput:\n{output}"
        
        return f"Command executed successfully.\nOutput:\n{output}"
    except subprocess.TimeoutExpired:
        return f"Error: Command execution timed out after {timeout} seconds"
    except Exception as e:
        return f"Error executing command: {str(e)}"

In [ ]:
# Execute a safe command
print(execute("ls -la"))

Command executed successfully.
Output:
total 1576
drwxr-xr-x@ 36 nathan  _lpoperator    1152 May  9 09:16 .
drwxr-xr-x@ 58 nathan  _lpoperator    1856 May  8 12:24 ..
drwxr-xr-x@  3 nathan  _lpoperator      96 May  9 08:12 .claude
drwxr-xr-x@ 15 nathan  _lpoperator     480 May  9 07:46 .git
drwxr-xr-x@  3 nathan  _lpoperator      96 Apr 10 14:32 .github
-rw-r--r--@  1 nathan  _lpoperator    1648 May  9 08:12 .gitignore
-rw-r--r--@  1 nathan  _lpoperator       0 May  8 14:29 .last_checked
-rw-r--r--@  1 nathan  _lpoperator  265817 May  8 14:29 00_core.ipynb
-rw-r--r--@  1 nathan  _lpoperator   78760 May  8 12:49 01_toolloop.ipynb
-rw-r--r--@  1 nathan  _lpoperator   75032 May  8 12:49 02_async.ipynb
-rw-r--r--@  1 nathan  _lpoperator   22180 May  8 12:49 03_text_editor.ipynb
-rw-r--r--@  1 nathan  _lpoperator   18042 May  9 14:52 04_bash_tools.ipynb
-rw-r--r--@  1 nathan  _lpoperator    4488 May  8 12:49 CHANGELOG.md
-rw-r--r--@  1 nathan  _lpoperator    3933 May  9 07:55 CLAUDE.md
-rw-

## File Finding

Find files matching a pattern with safety limits.

In [ ]:
# | export
def find_files(
    pattern: str,                                    # File pattern to match (e.g. "*.py" or "**/*.md")
    path: str = ".",                                 # Directory to search in
    max_depth: int = 3,                              # Maximum directory depth to search
    max_results: int = 100                           # Maximum number of results to return
) -> str:
    """Find files matching a pattern with reasonable limits.
    
    Args:
        pattern: File pattern to match (e.g. "*.py" or "**/*.md")
        path: Directory to search in
        max_depth: Maximum directory depth to search
        max_results: Maximum number of results to return
    
    Returns:
        List of matching files or error message
    """
    try:
        p = Path(path).expanduser().resolve()
        if not p.exists():
            return f"Error: Path '{p}' does not exist"
        
        # Handle max_depth for recursive patterns
        if "**" in pattern and max_depth > 0:
            # Some glob implementations support depth limiting syntax
            glob_pattern = pattern.replace("**", f"**[0-{max_depth}]")
        else:
            glob_pattern = pattern
            
        # Find matching files
        files = list(p.glob(glob_pattern))
        
        # Apply limits
        total_found = len(files)
        files = files[:max_results]
        
        if not files:
            return f"No files matching '{pattern}' found in {p}"
            
        # Format output
        results = '\n'.join(str(f) for f in files)
        
        # Add note if results were limited
        if total_found > max_results:
            results += f"\n\nNote: Showing {max_results} of {total_found} matches. Use a more specific pattern to narrow results."
            
        return results
    except Exception as e:
        return f"Error finding files: {str(e)}"

In [ ]:
# Find Python files in the current directory
find_files("*.py")

'/Users/nathan/aai-ws/claudette/setup.py'

## File Content Search

Search for patterns in files with safety limits.

In [ ]:
# | export
def grep_files(
    pattern: str,                                    # Pattern to search for
    path: str,                                       # File or directory to search in
    recursive: bool = False,                         # Whether to search recursively
    file_pattern: str = "*",                         # File pattern to limit the search (e.g. "*.py")
    max_results: int = 100                           # Maximum number of matches to return
) -> str:
    """Search for a pattern in files using the grep command.
    
    Args:
        pattern: Pattern to search for
        path: File or directory to search in
        recursive: Whether to search recursively
        file_pattern: File pattern to limit the search (e.g. "*.py")
        max_results: Maximum number of matches to return
    
    Returns:
        Matching lines with file and line numbers
    """
    try:
        p = Path(path).expanduser().resolve()
        if not p.exists():
            return f"Error: Path '{p}' does not exist"
        
        # Build the grep command with appropriate options
        # -n: show line numbers, -H: show filenames
        cmd = ["grep", "-n", "-H"]
        
        # Add recursive flag if requested
        if recursive:
            cmd.append("-r")
        
        # Add the pattern to search for
        cmd.append(pattern)
        
        # Add the path to search
        cmd.append(str(p))
        
        # Add file pattern if not searching all files
        if file_pattern != "*":
            cmd.extend(["--include", file_pattern])
        
        # Run the command with timeout
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            timeout=30
        )
        
        # If grep returns with exit code 1, it means no matches were found (but not an error)
        if result.returncode == 1 and not result.stderr:
            return f"No matches found for '{pattern}' in {path}"
        
        # If grep returns with another non-zero exit code, it's an error
        if result.returncode != 0 and result.returncode != 1:
            return f"Error searching files: {result.stderr}"
        
        # Split the output into lines and limit the results
        lines = result.stdout.strip().split("\n")
        total_matches = len(lines) if lines and lines[0] else 0
        
        if total_matches == 0:
            return f"No matches found for '{pattern}' in {path}"
        
        # Limit the number of results
        limited_lines = lines[:max_results]
        output = "\n".join(limited_lines)
        
        # Add note if results were limited
        if total_matches > max_results:
            output += f"\n\nNote: Showing {max_results} of {total_matches} matches. Use a more specific pattern to narrow results."
        
        return output
    except subprocess.TimeoutExpired:
        return "Error: Search timed out. Try a more specific pattern or smaller directory."
    except Exception as e:
        return f"Error searching files: {str(e)}"

In [ ]:
# Search for a pattern in Python files
print(grep_files("def", "./claudette", recursive=True, max_results=10))

/Users/nathan/aai-ws/claudette/claudette/toolloop.py:18:def toolloop(self:Chat,
/Users/nathan/aai-ws/claudette/claudette/toolloop.py:48:async def toolloop(self:AsyncChat,
/Users/nathan/aai-ws/claudette/claudette/core.py:84:def can_stream(m): return m in has_streaming_models
/Users/nathan/aai-ws/claudette/claudette/core.py:85:def can_set_system_prompt(m): return m in has_system_prompt_models
/Users/nathan/aai-ws/claudette/claudette/core.py:86:def can_set_temperature(m): return m in has_temperature_models
/Users/nathan/aai-ws/claudette/claudette/core.py:87:def can_use_extended_thinking(m): return m in has_extended_thinking_models
/Users/nathan/aai-ws/claudette/claudette/core.py:90:def find_block(r:abc.Mapping, # The message to look in
/Users/nathan/aai-ws/claudette/claudette/core.py:98:def _repr_markdown_(self:(Message)):
/Users/nathan/aai-ws/claudette/claudette/core.py:110:def usage(inp=0, # input tokens
/Users/nathan/aai-ws/claudette/claudette/core.py:119:def _dgetattr(o,s,d): 

Note: 

In [ ]:
# Search for a pattern in Python files
print(grep_files("def", "./claudette", recursive=True, file_pattern="*.py", max_results=10))

/Users/nathan/aai-ws/claudette/claudette/toolloop.py:18:def toolloop(self:Chat,
/Users/nathan/aai-ws/claudette/claudette/toolloop.py:48:async def toolloop(self:AsyncChat,
/Users/nathan/aai-ws/claudette/claudette/core.py:84:def can_stream(m): return m in has_streaming_models
/Users/nathan/aai-ws/claudette/claudette/core.py:85:def can_set_system_prompt(m): return m in has_system_prompt_models
/Users/nathan/aai-ws/claudette/claudette/core.py:86:def can_set_temperature(m): return m in has_temperature_models
/Users/nathan/aai-ws/claudette/claudette/core.py:87:def can_use_extended_thinking(m): return m in has_extended_thinking_models
/Users/nathan/aai-ws/claudette/claudette/core.py:90:def find_block(r:abc.Mapping, # The message to look in
/Users/nathan/aai-ws/claudette/claudette/core.py:98:def _repr_markdown_(self:(Message)):
/Users/nathan/aai-ws/claudette/claudette/core.py:110:def usage(inp=0, # input tokens
/Users/nathan/aai-ws/claudette/claudette/core.py:119:def _dgetattr(o,s,d): 

Note: 

## Examples

Here's how to use the bash tools with Claudette:

In [ ]:
# Example usage with Chat
from claudette import *

# Or using individual functions as tools
chat = Chat(models[1], tools=[execute, find_files, grep_files])

In [ ]:
chat.toolloop('Please use the provided tools to understand the contents of the current directory.')

Based on my exploration, here's a summary of the contents of the current directory:

## Project Overview: Claudette

This directory contains the source code and documentation for **Claudette**, which is a high-level wrapper for Anthropic's Python SDK for interacting with Claude AI models.

### Key Components:

1. **Main Python Modules**:
   - `core.py`: Contains the main `Chat` class for stateful dialogs with Claude
   - `toolloop.py`: Implements tool use functionality for Claude
   - `asink.py`: Provides async versions of the main classes
   - `text_editor.py`: Implements text editing capabilities
   - `bash_tools.py`: Provides bash/shell integration tools

2. **Documentation and Examples**:
   - Jupyter notebooks (00_core.ipynb, 01_toolloop.ipynb, etc.) that serve as both documentation and source code
   - README.md and CLAUDE.md with project overview and usage instructions

3. **Project Structure**:
   - The project follows the "literate nbdev" approach, where the source code is written as Jupyter notebooks with detailed explanations
   - The Python modules are generated from these notebooks

### Main Features:

1. A `Chat` class for stateful dialogs with Claude
2. Support for "prefill" to control Claude's initial response words
3. Convenient image handling capabilities
4. Simple tool use API integration
5. Async support for non-blocking operations
6. Text editing capabilities
7. Bash/shell integration tools

### Development Files:
- Configuration files for development (.gitignore, pyproject.toml, etc.)
- Build and packaging files (setup.py, MANIFEST.in)

The project is designed to simplify working with Claude's API while maintaining full control, reducing boilerplate code that would otherwise be needed with the base SDK.

<details>

- id: `msg_019DqzCReerDwew9496SBjmv`
- content: `[{'citations': None, 'text': 'Based on my exploration, here\'s a summary of the contents of the current directory:\n\n## Project Overview: Claudette\n\nThis directory contains the source code and documentation for **Claudette**, which is a high-level wrapper for Anthropic\'s Python SDK for interacting with Claude AI models.\n\n### Key Components:\n\n1. **Main Python Modules**:\n   - `core.py`: Contains the main `Chat` class for stateful dialogs with Claude\n   - `toolloop.py`: Implements tool use functionality for Claude\n   - `asink.py`: Provides async versions of the main classes\n   - `text_editor.py`: Implements text editing capabilities\n   - `bash_tools.py`: Provides bash/shell integration tools\n\n2. **Documentation and Examples**:\n   - Jupyter notebooks (00_core.ipynb, 01_toolloop.ipynb, etc.) that serve as both documentation and source code\n   - README.md and CLAUDE.md with project overview and usage instructions\n\n3. **Project Structure**:\n   - The project follows the "literate nbdev" approach, where the source code is written as Jupyter notebooks with detailed explanations\n   - The Python modules are generated from these notebooks\n\n### Main Features:\n\n1. A `Chat` class for stateful dialogs with Claude\n2. Support for "prefill" to control Claude\'s initial response words\n3. Convenient image handling capabilities\n4. Simple tool use API integration\n5. Async support for non-blocking operations\n6. Text editing capabilities\n7. Bash/shell integration tools\n\n### Development Files:\n- Configuration files for development (.gitignore, pyproject.toml, etc.)\n- Build and packaging files (setup.py, MANIFEST.in)\n\nThe project is designed to simplify working with Claude\'s API while maintaining full control, reducing boilerplate code that would otherwise be needed with the base SDK.', 'type': 'text'}]`
- model: `claude-3-7-sonnet-20250219`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 13525, 'output_tokens': 426}`
- steps: `[{'role': 'assistant', 'content': [{'citations': None, 'text': "I'll help you understand the contents of the current directory. Let me use the tools to explore what's available.\n\nFirst, let's list the contents of the current directory:", 'type': 'text'}, {'id': 'toolu_017RnKgaK6W9BDVs9Wz4cMhG', 'input': {'command': 'ls -la'}, 'name': 'execute', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_017RnKgaK6W9BDVs9Wz4cMhG', 'content': 'Command executed successfully.\nOutput:\ntotal 1576\ndrwxr-xr-x@ 36 nathan  _lpoperator    1152 May  9 09:16 \x1b[34m.\x1b[m\x1b[m\ndrwxr-xr-x@ 58 nathan  _lpoperator    1856 May  8 12:24 \x1b[34m..\x1b[m\x1b[m\ndrwxr-xr-x@  3 nathan  _lpoperator      96 May  9 08:12 \x1b[34m.claude\x1b[m\x1b[m\ndrwxr-xr-x@ 15 nathan  _lpoperator     480 May  9 07:46 \x1b[34m.git\x1b[m\x1b[m\ndrwxr-xr-x@  3 nathan  _lpoperator      96 Apr 10 14:32 \x1b[34m.github\x1b[m\x1b[m\n-rw-r--r--@  1 nathan  _lpoperator    1648 May  9 08:12 .gitignore\n-rw-r--r--@  1 nathan  _lpoperator       0 May  8 14:29 .last_checked\n-rw-r--r--@  1 nathan  _lpoperator  265817 May  8 14:29 00_core.ipynb\n-rw-r--r--@  1 nathan  _lpoperator   78760 May  8 12:49 01_toolloop.ipynb\n-rw-r--r--@  1 nathan  _lpoperator   75032 May  8 12:49 02_async.ipynb\n-rw-r--r--@  1 nathan  _lpoperator   22180 May  8 12:49 03_text_editor.ipynb\n-rw-r--r--@  1 nathan  _lpoperator   18042 May  9 14:52 04_bash_tools.ipynb\n-rw-r--r--@  1 nathan  _lpoperator    4488 May  8 12:49 CHANGELOG.md\n-rw-r--r--@  1 nathan  _lpoperator    3933 May  9 07:55 CLAUDE.md\n-rw-r--r--@  1 nathan  _lpoperator      20 Apr 10 14:32 CNAME\n-rw-r--r--@  1 nathan  _lpoperator   11357 Apr 10 14:32 LICENSE\n-rw-r--r--@  1 nathan  _lpoperator     111 Apr 10 14:32 MANIFEST.in\n-rw-r--r--@  1 nathan  _lpoperator   30803 Apr 10 14:32 README.md\n-rw-r--r--@  1 nathan  _lpoperator   29031 Apr 10 14:32 README.txt\ndrwxr-xr-x@ 36 nathan  _lpoperator    1152 May  1 11:49 \x1b[34m_proc\x1b[m\x1b[m\n-rw-r--r--@  1 nathan  _lpoperator     635 Apr 10 14:32 _quarto.yml\n-rw-r--r--@  1 nathan  _lpoperator    2357 Apr 10 14:32 apilist.txt\ndrwxr-xr-x@ 10 nathan  _lpoperator     320 May  9 09:12 \x1b[34mclaudette\x1b[m\x1b[m\ndrwxr-xr-x@  9 nathan  _lpoperator     288 May  1 23:26 \x1b[34mclaudette.egg-info\x1b[m\x1b[m\n-rw-r--r--@  1 nathan  _lpoperator   94036 Apr 10 14:32 index.ipynb\n-rw-r--r--@  1 nathan  _lpoperator   65708 Apr 10 14:32 llms-ctx-full.txt\n-rw-r--r--@  1 nathan  _lpoperator   30093 Apr 10 14:32 llms-ctx.txt\n-rw-r--r--@  1 nathan  _lpoperator    1452 Apr 10 14:32 llms.txt\n-rw-r--r--@  1 nathan  _lpoperator     224 May  1 11:49 nbdev.yml\n-rw-r--r--@  1 nathan  _lpoperator     416 May  8 14:29 pyproject.toml\ndrwxr-xr-x@  3 nathan  _lpoperator      96 Apr 10 14:32 \x1b[34msamples\x1b[m\x1b[m\n-rw-r--r--@  1 nathan  _lpoperator     939 May  8 12:49 settings.ini\n-rw-r--r--@  1 nathan  _lpoperator    2596 Apr 10 14:32 setup.py\n-rw-r--r--@  1 nathan  _lpoperator     154 May  1 11:49 sidebar.yml\n-rw-r--r--@  1 nathan  _lpoperator     701 Apr 10 14:32 styles.css\ndrwxr-xr-x@  3 nathan  _lpoperator      96 Apr 10 14:32 \x1b[34mtools\x1b[m\x1b[m\n'}]}, {'role': 'assistant', 'content': [{'citations': None, 'text': "Now, let's look at the main directories to understand the project structure better:", 'type': 'text'}, {'id': 'toolu_01YaDeuEPyVJ4Fqv334RYie3', 'input': {'pattern': '*.py', 'path': 'claudette', 'max_depth': 2}, 'name': 'find_files', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01YaDeuEPyVJ4Fqv334RYie3', 'content': '/Users/nathan/aai-ws/claudette/claudette/toolloop.py\n/Users/nathan/aai-ws/claudette/claudette/_modidx.py\n/Users/nathan/aai-ws/claudette/claudette/__init__.py\n/Users/nathan/aai-ws/claudette/claudette/core.py\n/Users/nathan/aai-ws/claudette/claudette/text_editor.py\n/Users/nathan/aai-ws/claudette/claudette/bash_tools.py\n/Users/nathan/aai-ws/claudette/claudette/asink.py'}]}, {'role': 'assistant', 'content': [{'citations': None, 'text': "Let's check the README.md to get an overview of the project:", 'type': 'text'}, {'id': 'toolu_01FFyt5NiBzNLznYYfZZh8ty', 'input': {'command': 'cat README.md'}, 'name': 'execute', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01FFyt5NiBzNLznYYfZZh8ty', 'content': 'Command executed successfully.\nOutput:\n# claudette\n\n\n<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->\n\n> **NB**: If you are reading this in GitHub’s readme, we recommend you\n> instead read the much more nicely formatted [documentation\n> format](https://claudette.answer.ai/) of this tutorial.\n\n*Claudette* is a wrapper for Anthropic’s [Python\nSDK](https://github.com/anthropics/anthropic-sdk-python).\n\nThe SDK works well, but it is quite low level – it leaves the developer\nto do a lot of stuff manually. That’s a lot of extra work and\nboilerplate! Claudette automates pretty much everything that can be\nautomated, whilst providing full control. Amongst the features provided:\n\n- A [`Chat`](https://claudette.answer.ai/core.html#chat) class that\n  creates stateful dialogs\n- Support for *prefill*, which tells Claude what to use as the first few\n  words of its response\n- Convenient image support\n- Simple and convenient support for Claude’s new Tool Use API.\n\nYou’ll need to set the `ANTHROPIC_API_KEY` environment variable to the\nkey provided to you by Anthropic in order to use this library.\n\nNote that this library is the first ever “literate nbdev” project. That\nmeans that the actual source code for the library is a rendered Jupyter\nNotebook which includes callout notes and tips, HTML tables and images,\ndetailed explanations, and teaches *how* and *why* the code is written\nthe way it is. Even if you’ve never used the Anthropic Python SDK or\nClaude API before, you should be able to read the source code. Click\n[Claudette’s Source](https://claudette.answer.ai/core.html) to read it,\nor clone the git repo and execute the notebook yourself to see every\nstep of the creation process in action. The tutorial below includes\nlinks to API details which will take you to relevant parts of the\nsource. The reason this project is a new kind of literal program is\nbecause we take seriously Knuth’s call to action, that we have a “*moral\ncommitment*” to never write an “*illiterate program*” – and so we have a\ncommitment to making literate programming and easy and pleasant\nexperience. (For more on this, see [this\ntalk](https://www.youtube.com/watch?v=rX1yGxJijsI) from Hamel Husain.)\n\n> “*Let us change our traditional attitude to the construction of\n> programs: Instead of imagining that our main task is to instruct a\n> **computer** what to do, let us concentrate rather on explaining to\n> **human beings** what we want a computer to do.*” Donald E. Knuth,\n> [Literate\n> Programming](https://www.cs.tufts.edu/~nr/cs257/archive/literate-programming/01-knuth-lp.pdf)\n> (1984)\n\n## Install\n\n``` sh\npip install claudette\n```\n\n## Getting started\n\nAnthropic’s Python SDK will automatically be installed with Claudette,\nif you don’t already have it.\n\n``` python\nimport os\n# os.environ[\'ANTHROPIC_LOG\'] = \'debug\'\n```\n\nTo print every HTTP request and response in full, uncomment the above\nline.\n\n``` python\nfrom claudette import *\n```\n\nClaudette only exports the symbols that are needed to use the library,\nso you can use `import *` to import them. Alternatively, just use:\n\n``` python\nimport claudette\n```\n\n…and then add the prefix `claudette.` to any usages of the module.\n\nClaudette provides `models`, which is a list of models currently\navailable from the SDK.\n\n``` python\nmodels\n```\n\n    [\'claude-3-opus-20240229\',\n     \'claude-3-7-sonnet-20250219\',\n     \'claude-3-5-sonnet-20241022\',\n     \'claude-3-haiku-20240307\',\n     \'claude-3-5-haiku-20241022\']\n\nFor these examples, we’ll use Sonnet 3.5, since it’s awesome!\n\n``` python\nmodel = models[1]\n```\n\n## Chat\n\nThe main interface to Claudette is the\n[`Chat`](https://claudette.answer.ai/core.html#chat) class, which\nprovides a stateful interface to Claude:\n\n``` python\nchat = Chat(model, sp="""You are a helpful and concise assistant.""")\nchat("I\'m Jeremy")\n```\n\nHello, Jeremy! How can I assist you today?\n\n<details>\n\n- id: `msg_01JKhMQx949VK4aVMGdh2x59`\n- content:\n  `[{\'citations\': None, \'text\': \'Hello, Jeremy! How can I assist you today?\', \'type\': \'text\'}]`\n- model: `claude-3-7-sonnet-20250219`\n- role: `assistant`\n- stop_reason: `end_turn`\n- stop_sequence: `None`\n- type: `message`\n- usage:\n  `{\'cache_creation_input_tokens\': 0, \'cache_read_input_tokens\': 0, \'input_tokens\': 19, \'output_tokens\': 14}`\n\n</details>\n\n``` python\nr = chat("What\'s my name?")\nr\n```\n\nYour name is Jeremy, as you mentioned in your introduction.\n\n<details>\n\n- id: `msg_01W5rc2j8EAQDFzZWfg1Dikx`\n- content:\n  `[{\'citations\': None, \'text\': \'Your name is Jeremy, as you mentioned in your introduction.\', \'type\': \'text\'}]`\n- model: `claude-3-7-sonnet-20250219`\n- role: `assistant`\n- stop_reason: `end_turn`\n- stop_sequence: `None`\n- type: `message`\n- usage:\n  `{\'cache_creation_input_tokens\': 0, \'cache_read_input_tokens\': 0, \'input_tokens\': 41, \'output_tokens\': 15}`\n\n</details>\n\n``` python\nr = chat("What\'s my name?")\nr\n```\n\nYour name is Jeremy.\n\n<details>\n\n- id: `msg_01CFQui4Zobt3WYs7qbUBvyh`\n- content:\n  `[{\'citations\': None, \'text\': \'Your name is Jeremy.\', \'type\': \'text\'}]`\n- model: `claude-3-7-sonnet-20250219`\n- role: `assistant`\n- stop_reason: `end_turn`\n- stop_sequence: `None`\n- type: `message`\n- usage:\n  `{\'cache_creation_input_tokens\': 0, \'cache_read_input_tokens\': 0, \'input_tokens\': 64, \'output_tokens\': 8}`\n\n</details>\n\nAs you see above, displaying the results of a call in a notebook shows\njust the message contents, with the other details hidden behind a\ncollapsible section. Alternatively you can `print` the details:\n\n``` python\nprint(r)\n```\n\n    Message(id=\'msg_01CFQui4Zobt3WYs7qbUBvyh\', content=[TextBlock(citations=None, text=\'Your name is Jeremy.\', type=\'text\')], model=\'claude-3-7-sonnet-20250219\', role=\'assistant\', stop_reason=\'end_turn\', stop_sequence=None, type=\'message\', usage=In: 64; Out: 8; Cache create: 0; Cache read: 0; Total: 72)\n\nClaude supports adding an extra `assistant` message at the end, which\ncontains the *prefill* – i.e.\xa0the text we want Claude to assume the\nresponse starts with. Let’s try it out:\n\n``` python\nchat("Concisely, what is the meaning of life?",\n     prefill=\'According to Douglas Adams,\')\n```\n\nAccording to Douglas Adams, 42. Philosophically, it’s often considered\nto be finding personal meaning, happiness, and fulfillment through\nrelationships, purpose, and experiences.\n\n<details>\n\n- id: `msg_01HtFZp54GYJdfopa4vweb1G`\n- content:\n  `[{\'citations\': None, \'text\': "According to Douglas Adams,  42. Philosophically, it\'s often considered to be finding personal meaning, happiness, and fulfillment through relationships, purpose, and experiences.", \'type\': \'text\'}]`\n- model: `claude-3-7-sonnet-20250219`\n- role: `assistant`\n- stop_reason: `end_turn`\n- stop_sequence: `None`\n- type: `message`\n- usage:\n  `{\'cache_creation_input_tokens\': 0, \'cache_read_input_tokens\': 0, \'input_tokens\': 92, \'output_tokens\': 34}`\n\n</details>\n\nYou can add `stream=True` to stream the results as soon as they arrive\n(although you will only see the gradual generation if you execute the\nnotebook yourself, of course!)\n\n``` python\nfor o in chat("Concisely, what book was that in?", prefill=\'It was in\', stream=True):\n    print(o, end=\'\')\n```\n\n    It was in "The Hitchhiker\'s Guide to the Galaxy" by Douglas Adams.\n\n### Async\n\nAlternatively, you can use\n[`AsyncChat`](https://claudette.answer.ai/async.html#asyncchat) (or\n[`AsyncClient`](https://claudette.answer.ai/async.html#asyncclient)) for\nthe async versions, e.g:\n\n``` python\nchat = AsyncChat(model)\nawait chat("I\'m Jeremy")\n```\n\nHello, Jeremy! It’s nice to meet you. How are you doing today? Is there\nsomething I can help you with or would you like to chat about something\nspecific?\n\n<details>\n\n- id: `msg_01PqUxGc9uiZVxy6EChRXcuy`\n- content:\n  `[{\'citations\': None, \'text\': "Hello, Jeremy! It\'s nice to meet you. How are you doing today? Is there something I can help you with or would you like to chat about something specific?", \'type\': \'text\'}]`\n- model: `claude-3-7-sonnet-20250219`\n- role: `assistant`\n- stop_reason: `end_turn`\n- stop_sequence: `None`\n- type: `message`\n- usage:\n  `{\'cache_creation_input_tokens\': 0, \'cache_read_input_tokens\': 0, \'input_tokens\': 10, \'output_tokens\': 38}`\n\n</details>\n\nRemember to use `async for` when streaming in this case:\n\n``` python\nasync for o in await chat("Concisely, what is the meaning of life?",\n                          prefill=\'According to Douglas Adams,\', stream=True):\n    print(o, end=\'\')\n```\n\n    According to Douglas Adams,  42. More seriously, the meaning of life is likely what you create through your relationships, pursuits, and values that bring you fulfillment and purpose.\n\n## Prompt caching\n\nIf you use `mk_msg(msg, cache=True)`, then the message is cached using\nClaude’s [prompt\ncaching](https://docs.anthropic.com/en/docs/build-with-claude/prompt-caching)\nfeature. For instance, here we use caching when asking about Claudette’s\nreadme file:\n\n``` python\nchat = Chat(model, sp="""You are a helpful and concise assistant.""")\n```\n\n``` python\nnbtxt = Path(\'README.txt\').read_text()\nmsg = f\'\'\'<README>\n{nbtxt}\n</README>\nIn brief, what is the purpose of this project based on the readme?\'\'\'\nr = chat(mk_msg(msg, cache=True))\nr\n```\n\nClaudette is a high-level wrapper for Anthropic’s Python SDK that\nautomates common tasks and provides additional functionality. Its main\nfeatures include:\n\n1.  A Chat class for stateful dialogs\n2.  Support for prefill (controlling Claude’s initial response words)\n3.  Convenient image handling\n4.  Simple tool use API integration\n\nThe project is notable for being the first “literate nbdev” project,\nmeaning its source code is written as a detailed, readable Jupyter\nNotebook that includes explanations, examples, and teaching material\nalongside the functional code.\n\nThe goal is to simplify working with Claude’s API while maintaining full\ncontrol, reducing boilerplate code and manual work that would otherwise\nbe needed with the base SDK.\n\n<details>\n\n- id: `msg_018FtfRtRxkWRdEK69qnASgC`\n- content:\n  `[{\'citations\': None, \'text\': \'Claudette is a high-level wrapper for Anthropic\\\'s Python SDK that automates common tasks and provides additional functionality. Its main features include:\\n\\n1. A Chat class for stateful dialogs\\n2. Support for prefill (controlling Claude\\\'s initial response words)\\n3. Convenient image handling\\n4. Simple tool use API integration\\n\\nThe project is notable for being the first "literate nbdev" project, meaning its source code is written as a detailed, readable Jupyter Notebook that includes explanations, examples, and teaching material alongside the functional code.\\n\\nThe goal is to simplify working with Claude\\\'s API while maintaining full control, reducing boilerplate code and manual work that would otherwise be needed with the base SDK.\', \'type\': \'text\'}]`\n- model: `claude-3-7-sonnet-20250219`\n- role: `assistant`\n- stop_reason: `end_turn`\n- stop_sequence: `None`\n- type: `message`\n- usage:\n  `{\'cache_creation_input_tokens\': 9287, \'cache_read_input_tokens\': 0, \'input_tokens\': 4, \'output_tokens\': 157}`\n\n</details>\n\nThe response records the a cache has been created using these input\ntokens:\n\n``` python\nprint(r.usage)\n```\n\n    Usage(cache_creation_input_tokens=9287, cache_read_input_tokens=0, input_tokens=4, output_tokens=157)\n\nWe can now ask a followup question in this chat:\n\n``` python\nr = chat(\'How does it make tool use more ergonomic?\')\nr\n```\n\nAccording to the README, Claudette makes tool use more ergonomic in\nseveral ways:\n\n1.  It uses docments to make Python function definitions more\n    user-friendly - each parameter and return value should have a type\n    and description\n\n2.  It handles the tool calling process automatically - when Claude\n    returns a tool_use message, Claudette manages calling the tool with\n    the provided parameters behind the scenes\n\n3.  It provides a `toolloop` method that can handle multiple tool calls\n    in a single step to solve more complex problems\n\n4.  It allows you to pass a list of tools to the Chat constructor and\n    optionally force Claude to always use a specific tool via\n    `tool_choice`\n\nHere’s a simple example from the README:\n\n``` python\ndef sums(\n    a:int,  # First thing to sum \n    b:int=1 # Second thing to sum\n) -> int: # The sum of the inputs\n    "Adds a + b."\n    print(f"Finding the sum of {a} and {b}")\n    return a + b\n\nchat = Chat(model, sp=sp, tools=[sums], tool_choice=\'sums\')\n```\n\nThis makes it much simpler compared to manually handling all the tool\nuse logic that would be required with the base SDK.\n\n<details>\n\n- id: `msg_011hEtdB23ptoEaWyHgk4m2k`\n- content:\n  `[{\'citations\': None, \'text\': \'According to the README, Claudette makes tool use more ergonomic in several ways:\\n\\n1. It uses docments to make Python function definitions more user-friendly - each parameter and return value should have a type and description\\n\\n2. It handles the tool calling process automatically - when Claude returns a tool_use message, Claudette manages calling the tool with the provided parameters behind the scenes\\n\\n3. It provides a`toolloop`method that can handle multiple tool calls in a single step to solve more complex problems\\n\\n4. It allows you to pass a list of tools to the Chat constructor and optionally force Claude to always use a specific tool via`tool_choice```` \\n\\nHere\\\'s a simple example from the README:\\n\\n```python\\ndef sums(\\n    a:int,  # First thing to sum \\n    b:int=1 # Second thing to sum\\n) -> int: # The sum of the inputs\\n    "Adds a + b."\\n    print(f"Finding the sum of {a} and {b}")\\n    return a + b\\n\\nchat = Chat(model, sp=sp, tools=[sums], tool_choice=\\\'sums\\\')\\n```\\n\\nThis makes it much simpler compared to manually handling all the tool use logic that would be required with the base SDK.\', \'type\': \'text\'}] ````\n- model: `claude-3-7-sonnet-20250219`\n- role: `assistant`\n- stop_reason: `end_turn`\n- stop_sequence: `None`\n- type: `message`\n- usage:\n  `{\'cache_creation_input_tokens\': 0, \'cache_read_input_tokens\': 9287, \'input_tokens\': 175, \'output_tokens\': 280}`\n\n</details>\n\nWe can see that this only used ~200 regular input tokens – the 7000+\ncontext tokens have been read from cache.\n\n``` python\nprint(r.usage)\n```\n\n    Usage(cache_creation_input_tokens=0, cache_read_input_tokens=9287, input_tokens=175, output_tokens=280)\n\n``` python\nchat.use\n```\n\n    In: 179; Out: 437; Cache create: 9287; Cache read: 9287; Total: 19190\n\n## Tool use\n\n[Tool use](https://docs.anthropic.com/claude/docs/tool-use) lets Claude\nuse external tools.\n\nWe use [docments](https://fastcore.fast.ai/docments.html) to make\ndefining Python functions as ergonomic as possible. Each parameter (and\nthe return value) should have a type, and a docments comment with the\ndescription of what it is. As an example we’ll write a simple function\nthat adds numbers together, and will tell us when it’s being called:\n\n``` python\ndef sums(\n    a:int,  # First thing to sum\n    b:int=1 # Second thing to sum\n) -> int: # The sum of the inputs\n    "Adds a + b."\n    print(f"Finding the sum of {a} and {b}")\n    return a + b\n```\n\nSometimes Claude will say something like “according to the `sums` tool\nthe answer is” – generally we’d rather it just tells the user the\nanswer, so we can use a system prompt to help with this:\n\n``` python\nsp = "Never mention what tools you use."\n```\n\nWe’ll get Claude to add up some long numbers:\n\n``` python\na,b = 604542,6458932\npr = f"What is {a}+{b}?"\npr\n```\n\n    \'What is 604542+6458932?\'\n\nTo use tools, pass a list of them to\n[`Chat`](https://claudette.answer.ai/core.html#chat):\n\n``` python\nchat = Chat(model, sp=sp, tools=[sums])\n```\n\nTo force Claude to always answer using a tool, set `tool_choice` to that\nfunction name. When Claude needs to use a tool, it doesn’t return the\nanswer, but instead returns a `tool_use` message, which means we have to\ncall the named tool with the provided parameters.\n\n``` python\nr = chat(pr, tool_choice=\'sums\')\nr\n```\n\n    Finding the sum of 604542 and 6458932\n\nToolUseBlock(id=‘toolu_01915nm3QDrQH1GSzVU7Xvqt’, input={‘a’: 604542,\n‘b’: 6458932}, name=‘sums’, type=‘tool_use’)\n\n<details>\n\n- id: `msg_01JP7GyAbS9Tp1UCYD9dwThU`\n- content:\n  `[{\'id\': \'toolu_01915nm3QDrQH1GSzVU7Xvqt\', \'input\': {\'a\': 604542, \'b\': 6458932}, \'name\': \'sums\', \'type\': \'tool_use\'}]`\n- model: `claude-3-7-sonnet-20250219`\n- role: `assistant`\n- stop_reason: `tool_use`\n- stop_sequence: `None`\n- type: `message`\n- usage:\n  `{\'cache_creation_input_tokens\': 0, \'cache_read_input_tokens\': 0, \'input_tokens\': 442, \'output_tokens\': 53}`\n\n</details>\n\nClaudette handles all that for us – we just call it again, and it all\nhappens automatically:\n\n``` python\nchat()\n```\n\nThe sum of 604542 and 6458932 is 7,063,474.\n\n<details>\n\n- id: `msg_01RDhvKv8wUn5pfC5p1CxTxz`\n- content:\n  `[{\'citations\': None, \'text\': \'The sum of 604542 and 6458932 is 7,063,474.\', \'type\': \'text\'}]`\n- model: `claude-3-7-sonnet-20250219`\n- role: `assistant`\n- stop_reason: `end_turn`\n- stop_sequence: `None`\n- type: `message`\n- usage:\n  `{\'cache_creation_input_tokens\': 0, \'cache_read_input_tokens\': 0, \'input_tokens\': 524, \'output_tokens\': 25}`\n\n</details>\n\nYou can see how many tokens have been used at any time by checking the\n`use` property. Note that (as of May 2024) tool use in Claude uses a\n*lot* of tokens, since it automatically adds a large system prompt.\n\n``` python\nchat.use\n```\n\n    In: 966; Out: 78; Cache create: 0; Cache read: 0; Total: 1044\n\nWe can do everything needed to use tools in a single step, by using\n[`Chat.toolloop`](https://claudette.answer.ai/toolloop.html#chat.toolloop).\nThis can even call multiple tools as needed solve a problem. For\nexample, let’s define a tool to handle multiplication:\n\n``` python\ndef mults(\n    a:int,  # First thing to multiply\n    b:int=1 # Second thing to multiply\n) -> int: # The product of the inputs\n    "Multiplies a * b."\n    print(f"Finding the product of {a} and {b}")\n    return a * b\n```\n\nNow with a single call we can calculate `(a+b)*2` – by passing\n`show_trace` we can see each response from Claude in the process:\n\n``` python\nchat = Chat(model, sp=sp, tools=[sums,mults])\npr = f\'Calculate ({a}+{b})*2\'\npr\n```\n\n    \'Calculate (604542+6458932)*2\'\n\n``` python\nchat.toolloop(pr, trace_func=print)\n```\n\n    Finding the sum of 604542 and 6458932\n    [{\'role\': \'user\', \'content\': \'Calculate (604542+6458932)*2\'}, {\'role\': \'assistant\', \'content\': [TextBlock(citations=None, text="I\'ll calculate (604542+6458932)*2 for you.\\n\\nFirst, let me add the numbers inside the parentheses:", type=\'text\'), ToolUseBlock(id=\'toolu_01SkhYBiVNDAbCPkX3be6X4F\', input={\'a\': 604542, \'b\': 6458932}, name=\'sums\', type=\'tool_use\')]}, {\'role\': \'user\', \'content\': [{\'type\': \'tool_result\', \'tool_use_id\': \'toolu_01SkhYBiVNDAbCPkX3be6X4F\', \'content\': \'7063474\'}]}]\n    Finding the product of 7063474 and 2\n    [{\'role\': \'assistant\', \'content\': [TextBlock(citations=None, text="Now, I\'ll multiply this result by 2:", type=\'text\'), ToolUseBlock(id=\'toolu_01XaagwWimhXuMt2oKMBttWv\', input={\'a\': 7063474, \'b\': 2}, name=\'mults\', type=\'tool_use\')]}, {\'role\': \'user\', \'content\': [{\'type\': \'tool_result\', \'tool_use_id\': \'toolu_01XaagwWimhXuMt2oKMBttWv\', \'content\': \'14126948\'}]}]\n    [{\'role\': \'assistant\', \'content\': [TextBlock(citations=None, text=\'The result of (604542+6458932)*2 is 14,126,948.\', type=\'text\')]}]\n\nThe result of (604542+6458932)\\*2 is 14,126,948.\n\n<details>\n\n- id: `msg_015TzicGDqJnYbYEmVyxPMvd`\n- content:\n  `[{\'citations\': None, \'text\': \'The result of (604542+6458932)*2 is 14,126,948.\', \'type\': \'text\'}]`\n- model: `claude-3-7-sonnet-20250219`\n- role: `assistant`\n- stop_reason: `end_turn`\n- stop_sequence: `None`\n- type: `message`\n- usage:\n  `{\'cache_creation_input_tokens\': 0, \'cache_read_input_tokens\': 0, \'input_tokens\': 752, \'output_tokens\': 25}`\n\n</details>\n\n## Structured data\n\nIf you just want the immediate result from a single tool, use\n[`Client.structured`](https://claudette.answer.ai/core.html#client.structured).\n\n``` python\ncli = Client(model)\n```\n\n``` python\ndef sums(\n    a:int,  # First thing to sum\n    b:int=1 # Second thing to sum\n) -> int: # The sum of the inputs\n    "Adds a + b."\n    print(f"Finding the sum of {a} and {b}")\n    return a + b\n```\n\n``` python\ncli.structured("What is 604542+6458932", sums)\n```\n\n    Finding the sum of 604542 and 6458932\n\n    [7063474]\n\nThis is particularly useful for getting back structured information,\ne.g:\n\n``` python\nclass President:\n    "Information about a president of the United States"\n    def __init__(self, \n                first:str, # first name\n                last:str, # last name\n                spouse:str, # name of spouse\n                years_in_office:str, # format: "{start_year}-{end_year}"\n                birthplace:str, # name of city\n                birth_year:int # year of birth, `0` if unknown\n        ):\n        assert re.match(r\'\\d{4}-\\d{4}\', years_in_office), "Invalid format: `years_in_office`"\n        store_attr()\n\n    __repr__ = basic_repr(\'first, last, spouse, years_in_office, birthplace, birth_year\')\n```\n\n``` python\ncli.structured("Provide key information about the 3rd President of the United States", President)\n```\n\n    [President(first=\'Thomas\', last=\'Jefferson\', spouse=\'Martha Wayles Skelton\', years_in_office=\'1801-1809\', birthplace=\'Shadwell, Virginia\', birth_year=1743)]\n\n## Images\n\nClaude can handle image data as well. As everyone knows, when testing\nimage APIs you have to use a cute puppy.\n\n``` python\nfn = Path(\'samples/puppy.jpg\')\ndisplay.Image(filename=fn, width=200)\n```\n\n<img src="index_files/figure-commonmark/cell-35-output-1.jpeg"\nwidth="200" />\n\nWe create a [`Chat`](https://claudette.answer.ai/core.html#chat) object\nas before:\n\n``` python\nchat = Chat(model)\n```\n\nClaudette expects images as a list of bytes, so we read in the file:\n\n``` python\nimg = fn.read_bytes()\n```\n\nPrompts to Claudette can be lists, containing text, images, or both, eg:\n\n``` python\nchat([img, "In brief, what color flowers are in this image?"])\n```\n\nThe image shows purple flowers (appears to be asters or similar purple\ndaisies) blooming next to a small Cavalier King Charles Spaniel puppy.\nThe puppy is resting on grass, and the purple flowers create a nice\ncontrast with the natural setting.\n\n<details>\n\n- id: `msg_01MEKVkFVbh1MHZwY2PNy8PQ`\n- content:\n  `[{\'citations\': None, \'text\': \'The image shows purple flowers (appears to be asters or similar purple daisies) blooming next to a small Cavalier King Charles Spaniel puppy. The puppy is resting on grass, and the purple flowers create a nice contrast with the natural setting.\', \'type\': \'text\'}]`\n- model: `claude-3-7-sonnet-20250219`\n- role: `assistant`\n- stop_reason: `end_turn`\n- stop_sequence: `None`\n- type: `message`\n- usage:\n  `{\'cache_creation_input_tokens\': 0, \'cache_read_input_tokens\': 0, \'input_tokens\': 110, \'output_tokens\': 60}`\n\n</details>\n\nThe image is included as input tokens.\n\n``` python\nchat.use\n```\n\n    In: 110; Out: 60; Cache create: 0; Cache read: 0; Total: 170\n\nAlternatively, Claudette supports creating a multi-stage chat with\nseparate image and text prompts. For instance, you can pass just the\nimage as the initial prompt (in which case Claude will make some general\ncomments about what it sees), and then follow up with questions in\nadditional prompts:\n\n``` python\nchat = Chat(model)\nchat(img)\n```\n\nThis is an adorable puppy with a white and brown coat, resting on some\ngrass. The puppy appears to be a Cavalier King Charles Spaniel or\nsimilar breed, with distinctive brown patches on its ears and around one\neye. The puppy has a sweet expression and is lying near some purple\nflowers, likely asters or similar small daisy-like blooms. The setting\ncreates a charming garden scene with the natural elements of grass,\nflowers, and what looks like a stone or wooden structure in the\nbackground. The combination of the cute puppy and the pretty flowers\nmakes for a very heartwarming image.\n\n<details>\n\n- id: `msg_01CXq96V9U2Jstf578znibrt`\n- content:\n  `[{\'citations\': None, \'text\': \'This is an adorable puppy with a white and brown coat, resting on some grass. The puppy appears to be a Cavalier King Charles Spaniel or similar breed, with distinctive brown patches on its ears and around one eye. The puppy has a sweet expression and is lying near some purple flowers, likely asters or similar small daisy-like blooms. The setting creates a charming garden scene with the natural elements of grass, flowers, and what looks like a stone or wooden structure in the background. The combination of the cute puppy and the pretty flowers makes for a very heartwarming image.\', \'type\': \'text\'}]`\n- model: `claude-3-7-sonnet-20250219`\n- role: `assistant`\n- stop_reason: `end_turn`\n- stop_sequence: `None`\n- type: `message`\n- usage:\n  `{\'cache_creation_input_tokens\': 0, \'cache_read_input_tokens\': 0, \'input_tokens\': 98, \'output_tokens\': 132}`\n\n</details>\n\n``` python\nchat(\'What direction is the puppy facing?\')\n```\n\nThe puppy is facing toward the camera/viewer, with its head slightly\nangled. You can see its face clearly with both eyes visible, and its\nfront paws are extended forward on the grass. The puppy’s body is\npositioned in a relaxed lying down pose with its face as the focal point\nof the image.\n\n<details>\n\n- id: `msg_01L6qPiQqqoBUTVCfm2nqg7S`\n- content:\n  `[{\'citations\': None, \'text\': "The puppy is facing toward the camera/viewer, with its head slightly angled. You can see its face clearly with both eyes visible, and its front paws are extended forward on the grass. The puppy\'s body is positioned in a relaxed lying down pose with its face as the focal point of the image.", \'type\': \'text\'}]`\n- model: `claude-3-7-sonnet-20250219`\n- role: `assistant`\n- stop_reason: `end_turn`\n- stop_sequence: `None`\n- type: `message`\n- usage:\n  `{\'cache_creation_input_tokens\': 0, \'cache_read_input_tokens\': 0, \'input_tokens\': 241, \'output_tokens\': 69}`\n\n</details>\n\n``` python\nchat(\'What color is it?\')\n```\n\nThe puppy has a white and brown coat. Its body is predominantly white,\nwhile it has rich brown/chestnut coloring on its ears and a patch around\none eye. This color pattern is typical of Cavalier King Charles\nSpaniels, particularly the Blenheim color variety, which features the\ncharacteristic white base with chestnut or reddish-brown markings.\n\n<details>\n\n- id: `msg_019z266CT4uu6UCXHsGZ5Qw9`\n- content:\n  `[{\'citations\': None, \'text\': \'The puppy has a white and brown coat. Its body is predominantly white, while it has rich brown/chestnut coloring on its ears and a patch around one eye. This color pattern is typical of Cavalier King Charles Spaniels, particularly the Blenheim color variety, which features the characteristic white base with chestnut or reddish-brown markings.\', \'type\': \'text\'}]`\n- model: `claude-3-7-sonnet-20250219`\n- role: `assistant`\n- stop_reason: `end_turn`\n- stop_sequence: `None`\n- type: `message`\n- usage:\n  `{\'cache_creation_input_tokens\': 0, \'cache_read_input_tokens\': 0, \'input_tokens\': 318, \'output_tokens\': 84}`\n\n</details>\n\nNote that the image is passed in again for every input in the dialog, so\nthat number of input tokens increases quickly with this kind of chat.\n(For large images, using prompt caching might be a good idea.)\n\n``` python\nchat.use\n```\n\n    In: 657; Out: 285; Cache create: 0; Cache read: 0; Total: 942\n\n## Extended Thinking\n\nClaude 3.7 Sonnet has enhanced reasoning capabilities for complex tasks.\nSee\n[docs](https://docs.anthropic.com/en/docs/build-with-claude/extended-thinking)\nfor more info.\n\nTo enable extended thinking simply specify the no. of thinking tokens\n`maxthinktok` you want to use.\n\n``` python\nchat = Chat(first(has_extended_thinking_models))\n```\n\n``` python\nchat(\'Write a sentence about Python!\', maxthinktok=1024)\n```\n\nPython is a versatile, high-level programming language known for its\nreadable syntax and wide application in data science, web development,\nartificial intelligence, and automation.\n\n<details>\n\n<summary>\n\nThinking\n</summary>\n\nI’ll write a simple, informative sentence about the Python programming\nlanguage. I should make it clear that I’m referring to the programming\nlanguage rather than the snake.\n</details>\n\n<details>\n\n- id: `msg_01Cxug4hSBsiDZ3KWfUkQdUF`\n- content:\n  `[{\'signature\': \'ErUBCkYIARgCIkDVNeCzqoRFz0iS5CTiMQA//IJi/R/cq25udwX71lATHFTFTAvaLG7RoQdLaI1cZppD6BTK4VDE30/O007g5DIeEgzZGw7VTS5rOS2jSEYaDDIK0RzCgTZc+bAlsiIw92rY7M6VJ6HtFk898McV6vfOfhcTJkoEBpHFy8WbqJ/HFq8qpwDRGF5HWS5zUSGmKh24Ex0wiBfWEtD/C0q3o02ulDfjJBhqaRfIdWGJjg==\', \'thinking\': "I\'ll write a simple, informative sentence about the Python programming language. I should make it clear that I\'m referring to the programming language rather than the snake.", \'type\': \'thinking\'}, {\'citations\': None, \'text\': \'Python is a versatile, high-level programming language known for its readable syntax and wide application in data science, web development, artificial intelligence, and automation.\', \'type\': \'text\'}]`\n- model: `claude-3-7-sonnet-20250219`\n- role: `assistant`\n- stop_reason: `end_turn`\n- stop_sequence: `None`\n- type: `message`\n- usage:\n  `{\'cache_creation_input_tokens\': 0, \'cache_read_input_tokens\': 0, \'input_tokens\': 41, \'output_tokens\': 77}`\n\n</details>\n\n``` python\nchat.use\n```\n\n    In: 41; Out: 77; Cache create: 0; Cache read: 0; Total: 118\n\n## Other model providers\n\nYou can also use 3rd party providers of Anthropic models, as shown here.\n\n### Amazon Bedrock\n\nThese are the models available through Bedrock:\n\n``` python\nmodels_aws\n```\n\n    [\'claude-3-5-haiku-20241022\',\n     \'claude-3-7-sonnet-20250219\',\n     \'anthropic.claude-3-opus-20240229-v1:0\',\n     \'anthropic.claude-3-5-sonnet-20241022-v2:0\']\n\nTo use them, call `AnthropicBedrock` with your access details, and pass\nthat to [`Client`](https://claudette.answer.ai/core.html#client):\n\n``` python\nfrom anthropic import AnthropicBedrock\n```\n\n``` python\nab = AnthropicBedrock(\n    aws_access_key=os.environ[\'AWS_ACCESS_KEY\'],\n    aws_secret_key=os.environ[\'AWS_SECRET_KEY\'],\n)\nclient = Client(models_aws[-1], ab)\n```\n\nNow create your [`Chat`](https://claudette.answer.ai/core.html#chat)\nobject passing this client to the `cli` parameter – and from then on,\neverything is identical to the previous examples.\n\n``` python\nchat = Chat(cli=client)\nchat("I\'m Jeremy")\n```\n\n### Google Vertex\n\nThese are the models available through Vertex:\n\n``` python\nmodels_goog\n```\n\nTo use them, call `AnthropicVertex` with your access details, and pass\nthat to [`Client`](https://claudette.answer.ai/core.html#client):\n\n``` python\nfrom anthropic import AnthropicVertex\nimport google.auth\n```\n\n``` python\nproject_id = google.auth.default()[1]\ngv = AnthropicVertex(project_id=project_id, region="us-east5")\nclient = Client(models_goog[-1], gv)\n```\n\n``` python\nchat = Chat(cli=client)\nchat("I\'m Jeremy")\n```\n\n## Extensions\n\n- [Pydantic Structured\n  Ouput](https://github.com/tom-pollak/claudette-pydantic)\n'}]}, {'role': 'assistant', 'content': [{'citations': None, 'text': "Let's also look at the project structure by examining the Python files in the main directory:", 'type': 'text'}, {'id': 'toolu_01UfwHbw938VnqVwze4oqZes', 'input': {'command': 'find claudette -name "*.py" -type f | sort'}, 'name': 'execute', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01UfwHbw938VnqVwze4oqZes', 'content': 'Command executed successfully.\nOutput:\nclaudette/__init__.py\nclaudette/_modidx.py\nclaudette/asink.py\nclaudette/bash_tools.py\nclaudette/core.py\nclaudette/text_editor.py\nclaudette/toolloop.py\n'}]}, {'role': 'assistant', 'content': [{'citations': None, 'text': "Let's examine the content of the main module files to understand their purpose:", 'type': 'text'}, {'id': 'toolu_01HP6TLSbrctUyw3ddN9zTmJ', 'input': {'command': 'cat claudette/__init__.py'}, 'name': 'execute', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01HP6TLSbrctUyw3ddN9zTmJ', 'content': 'Command executed successfully.\nOutput:\n__version__ = "0.1.8"\nfrom .core import *\nfrom .toolloop import *\nfrom .asink import *\nfrom .text_editor import *\nfrom .bash_tools import *\n\n'}]}, {'role': 'assistant', 'content': [{'citations': None, 'text': "Let's check the notebook files to understand the examples:", 'type': 'text'}, {'id': 'toolu_01Fga8S86ZGkCLiFoAfT2Cq8', 'input': {'command': 'ls -la *.ipynb'}, 'name': 'execute', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01Fga8S86ZGkCLiFoAfT2Cq8', 'content': 'Command executed successfully.\nOutput:\n-rw-r--r--@ 1 nathan  _lpoperator  265817 May  8 14:29 00_core.ipynb\n-rw-r--r--@ 1 nathan  _lpoperator   78760 May  8 12:49 01_toolloop.ipynb\n-rw-r--r--@ 1 nathan  _lpoperator   75032 May  8 12:49 02_async.ipynb\n-rw-r--r--@ 1 nathan  _lpoperator   22180 May  8 12:49 03_text_editor.ipynb\n-rw-r--r--@ 1 nathan  _lpoperator   19659 May  9 14:54 04_bash_tools.ipynb\n-rw-r--r--@ 1 nathan  _lpoperator   94036 Apr 10 14:32 index.ipynb\n'}]}, {'role': 'assistant', 'content': [{'citations': None, 'text': 'Based on my exploration, here\'s a summary of the contents of the current directory:\n\n## Project Overview: Claudette\n\nThis directory contains the source code and documentation for **Claudette**, which is a high-level wrapper for Anthropic\'s Python SDK for interacting with Claude AI models.\n\n### Key Components:\n\n1. **Main Python Modules**:\n   - `core.py`: Contains the main `Chat` class for stateful dialogs with Claude\n   - `toolloop.py`: Implements tool use functionality for Claude\n   - `asink.py`: Provides async versions of the main classes\n   - `text_editor.py`: Implements text editing capabilities\n   - `bash_tools.py`: Provides bash/shell integration tools\n\n2. **Documentation and Examples**:\n   - Jupyter notebooks (00_core.ipynb, 01_toolloop.ipynb, etc.) that serve as both documentation and source code\n   - README.md and CLAUDE.md with project overview and usage instructions\n\n3. **Project Structure**:\n   - The project follows the "literate nbdev" approach, where the source code is written as Jupyter notebooks with detailed explanations\n   - The Python modules are generated from these notebooks\n\n### Main Features:\n\n1. A `Chat` class for stateful dialogs with Claude\n2. Support for "prefill" to control Claude\'s initial response words\n3. Convenient image handling capabilities\n4. Simple tool use API integration\n5. Async support for non-blocking operations\n6. Text editing capabilities\n7. Bash/shell integration tools\n\n### Development Files:\n- Configuration files for development (.gitignore, pyproject.toml, etc.)\n- Build and packaging files (setup.py, MANIFEST.in)\n\nThe project is designed to simplify working with Claude\'s API while maintaining full control, reducing boilerplate code that would otherwise be needed with the base SDK.', 'type': 'text'}]}]`

</details>